In [1]:
import spike.spike_analysis.spike_collection as sc
import spike.spike_analysis.spike_recording as sr
import spike.spike_analysis.firing_rate_calculations as fr
import spike.spike_analysis.normalization as norm
import spike.spike_analysis.single_cell as single_cell
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import behavior.boris_extraction as boris
import matplotlib.pyplot as plt
import behavior.behavioral_epoch_tools as bet

In [31]:
# Initiate the spike collection object
object_collection = sc.SpikeCollection(path = r"C:\Users\zhaoz\Desktop\Research\data", sampling_rate= 20000)

loading  22_object_merged.rec
loading  23_object_merged.rec
23_object_merged.rec has no good units
and will not be included in the collection
loading  31_object_merged.rec
loading  32_object_merged.rec
loading  41_object_merged.rec
loading  44_object_merged.rec
Please assign event dictionaries to each recording
as recording.event_dict
event_dict = {event name(str): np.array[[start(ms), stop(ms)]...]
Please assign subjects to each recording as recording.subject


In [32]:
# Print the recordings detail
object_collection.recording_details()

Recording Details:

Recording: 22_object_merged.rec
  Subject: Unknown
  Number of Good Units: 14
  Recording Length: 30.18 minutes
  Events per Event Type: {}

Recording: 31_object_merged.rec
  Subject: Unknown
  Number of Good Units: 1
  Recording Length: 30.51 minutes
  Events per Event Type: {}

Recording: 32_object_merged.rec
  Subject: Unknown
  Number of Good Units: 9
  Recording Length: 30.52 minutes
  Events per Event Type: {}

Recording: 41_object_merged.rec
  Subject: Unknown
  Number of Good Units: 23
  Recording Length: 30.51 minutes
  Events per Event Type: {}

Recording: 44_object_merged.rec
  Subject: Unknown
  Number of Good Units: 4
  Recording Length: 30.16 minutes
  Events per Event Type: {}



In [33]:
event_dict = {}
subject_dict = {}
for file in os.listdir(r"C:\Users\zhaoz\Desktop\Research\data\aggregated_csvs"):
    recording_name_parts = (file.split(".")[0]).split("_")[:-1]
    recording_name = "_".join(recording_name_parts) + "_merged.rec"
    if file.endswith(".csv"):
        behavior = ['facial sniffing', 'anogenital sniffing']
        rec_event_dict = {}
        min_iti = 0.5
        min_bout = 0.3
        behavior_path = r"C:\Users\zhaoz\Desktop\Research\data\aggregated_csvs\\" + file
        behavior_df = pd.read_csv(behavior_path)
        subject_only_sniffing = boris.get_behavior_bouts(behavior_df,
                                                         subject = ['subject'],
                                                         behavior = behavior,
                                                         min_iti = min_iti,
                                                         min_bout = min_bout)
        agent_only_sniffing = boris.get_behavior_bouts(behavior_df,
                                                         subject = ['social_agent'],
                                                         behavior = behavior,
                                                         min_iti = min_iti,
                                                         min_bout = min_bout)
        duplicate_dict = bet.overlapping_events(eventA = subject_only_sniffing,
                                                eventB = agent_only_sniffing,
                                                overlap_threshold = .5,
                                                mode = 'duplicate',
                                                return_nonoverlap = True)
        rec_event_dict['subject only sniffing'] = duplicate_dict['eventA']
        rec_event_dict['social agent only sniffing'] = duplicate_dict['eventB']
        rec_event_dict['reciprocal sniffing'] = duplicate_dict['event1']
        event_dict[recording_name] = rec_event_dict
        subject = recording_name_parts[0]
        subject_dict[recording_name] = subject

print(event_dict.keys())
print(subject_dict)

dict_keys(['22_object_merged.rec', '23_object_merged.rec', '31_object_merged.rec', '41_object_merged.rec', '44_object_merged.rec'])
{'22_object_merged.rec': '22', '23_object_merged.rec': '23', '31_object_merged.rec': '31', '41_object_merged.rec': '41', '44_object_merged.rec': '44'}


In [34]:
object_collection = sc.SpikeCollection(path = r"C:\Users\zhaoz\Desktop\Research\data",
									   event_dict = event_dict,
									   subject_dict = subject_dict,
									   sampling_rate = 20000)
object_collection.recording_details()

loading  22_object_merged.rec
loading  23_object_merged.rec
23_object_merged.rec has no good units
and will not be included in the collection
loading  31_object_merged.rec
loading  32_object_merged.rec
32_object_merged.rec not found in subject dict
32_object_merged.rec not found in event dict
loading  41_object_merged.rec
loading  44_object_merged.rec
Recording Details:

Recording: 22_object_merged.rec
  Subject: 22
  Number of Good Units: 14
  Recording Length: 30.18 minutes
  Events per Event Type: {'subject only sniffing': 50, 'social agent only sniffing': 28, 'reciprocal sniffing': 1}

Recording: 31_object_merged.rec
  Subject: 31
  Number of Good Units: 1
  Recording Length: 30.51 minutes
  Events per Event Type: {'subject only sniffing': 7, 'social agent only sniffing': 35, 'reciprocal sniffing': 4}

Recording: 32_object_merged.rec
  Subject: Unknown
  Number of Good Units: 9
  Recording Length: 30.52 minutes
  Events per Event Type: {}

Recording: 41_object_merged.rec
  Subject: